# Citi Bike Challenge: Data Cleaning and Transformation

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime as dt
from pathlib import Path

In [2]:
# Function to read csv and return a dataframe
def get_month_data(month, year):
    path = 'resources/JC-' + str(year) + month + '-citibike-tripdata.csv'
    month_df = pd.read_csv(path, index_col=False)
    return month_df

In [3]:
def get_year_data(year):
    
    # list to hold dataframes for each month
    list = []
    
    # Loop through months to import each csv file
    for month in range(1, 13):
        
        # Call function to get data from csv (format month as two digits to match filename)
        df = get_month_data(f"{month:02}", year)
        
        # Modify df to mirror csvs after 01/2021
        if (year < 2021) or (year == 2021 and month == 1):
            df.drop(columns=['tripduration', 'bikeid', 'birth year', 'gender'], inplace = True)
            df['ride_id'] = 'NA'
            df['rideable_type'] = 'NA'
            df['member_casual'] = df['usertype']
            df.loc[df['usertype'] == 'Subscriber', 'member_casual'] = 'member'
            df.loc[df['usertype'] == 'Customer', 'member_casual'] = 'casual'
            columns = {'starttime': 'started_at', 'stoptime': 'ended_at',
                       'start station name': 'start_station_name',
                       'start station latitude': 'start_lat', 'start station longitude': 'start_lng',
                       'end station name': 'end_station_name',
                       'end station latitude': 'end_lat', 'end station longitude': 'end_lng' 
                      }   
            df.rename(columns=columns, inplace=True)
            df = df[['ride_id', 'rideable_type', 'started_at', 'ended_at',
                     'start_station_name', 'end_station_name',
                     'start_lat', 'start_lng', 'end_lat', 'end_lng',
                     'member_casual',
                    ]]
        else:
            df.drop(columns=['start_station_id', 'end_station_id'], inplace=True)
        
        df['started_at'] = pd.to_datetime(df['started_at'], errors='raise')
        df['ended_at'] = pd.to_datetime(df['ended_at'], errors='raise')   
        
        # Break loop as there is only data up to 6/2023
        if (year == 2023) and (month == 6):
            break
        
        # append month dataframe to list
        list.append(df)
        
    return list

In [4]:
# Create list to store dataframes for each month
citi_bike_list = []

# Loop through years to import data
for year in range(2020, 2024):
    print(f'Gathering data for {year}')   
    citi_bike_list = citi_bike_list + get_year_data(year)

Gathering data for 2020
Gathering data for 2021
Gathering data for 2022
Gathering data for 2023


In [5]:
citi_bike_df = pd.concat(citi_bike_list, axis=0, ignore_index=True)
citi_bike_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual
0,NA,NA,2020-01-01 00:04:50.192,2020-01-01 00:08:37.037,Grove St PATH,Newark Ave,40.719586,-74.043117,40.721525,-74.046305,member
1,NA,NA,2020-01-01 00:16:01.670,2020-01-01 00:22:19.080,Grove St PATH,Brunswick & 6th,40.719586,-74.043117,40.726012,-74.050389,member
2,NA,NA,2020-01-01 00:17:33.877,2020-01-01 00:22:22.442,Grove St PATH,Brunswick & 6th,40.719586,-74.043117,40.726012,-74.050389,casual
3,NA,NA,2020-01-01 00:32:05.902,2020-01-01 00:39:21.066,Sip Ave,Astor Place,40.730897,-74.063913,40.719282,-74.071262,casual
4,NA,NA,2020-01-01 00:46:19.678,2020-01-01 00:50:11.344,Grove St PATH,Marin Light Rail,40.719586,-74.043117,40.714584,-74.042817,member
...,...,...,...,...,...,...,...,...,...,...,...
2223775,9549398286BB59B5,docked_bike,2023-05-27 13:24:51.000,2023-05-27 13:51:18.000,South Waterfront Walkway - Sinatra Dr & 1 St,Newport Pkwy,40.736982,-74.027781,40.728745,-74.032108,casual
2223776,AB0790683044D89E,classic_bike,2023-05-18 17:12:07.000,2023-05-18 17:25:02.000,South Waterfront Walkway - Sinatra Dr & 1 St,Warren St,40.736982,-74.027781,40.721124,-74.038051,member
2223777,68226FA538A405CA,classic_bike,2023-05-05 18:44:57.000,2023-05-05 18:56:15.000,South Waterfront Walkway - Sinatra Dr & 1 St,Warren St,40.736982,-74.027781,40.721124,-74.038051,member
2223778,CABAC21C0F0140A5,classic_bike,2023-05-16 11:56:11.000,2023-05-16 12:04:04.000,Columbus Drive,Newport Pkwy,40.718355,-74.038914,40.728745,-74.032108,member


## Additional Data Cleaning

In [6]:
citi_bike_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2223780 entries, 0 to 2223779
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   ride_id             2223780 non-null  object        
 1   rideable_type       2223780 non-null  object        
 2   started_at          2223780 non-null  datetime64[ns]
 3   ended_at            2223780 non-null  datetime64[ns]
 4   start_station_name  2223767 non-null  object        
 5   end_station_name    2216045 non-null  object        
 6   start_lat           2223780 non-null  float64       
 7   start_lng           2223780 non-null  float64       
 8   end_lat             2219216 non-null  float64       
 9   end_lng             2219216 non-null  float64       
 10  member_casual       2223780 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(5)
memory usage: 186.6+ MB


In [7]:
# Reviewing rows with missing data
missing_data = citi_bike_df[citi_bike_df.isna().any(axis=1)]
missing_data

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual
348513,DF695B318E00BB00,docked_bike,2021-02-07 15:44:49,2021-02-08 16:44:41,Communipaw & Berry Lane,NaN,40.714358,-74.066610,NaN,NaN,member
349194,7B044F46E7E17711,docked_bike,2021-02-27 18:24:58,2021-02-27 19:47:28,Glenwood Ave,NaN,40.727551,-74.071060,NaN,NaN,member
349196,FF34C7420CE65E80,docked_bike,2021-02-19 20:08:02,2021-02-20 11:54:07,Marin Light Rail,NaN,40.714584,-74.042817,NaN,NaN,casual
349197,56108F9162464B62,docked_bike,2021-02-13 17:33:20,2021-02-13 22:34:42,Monmouth and 6th,NaN,40.725685,-74.048790,NaN,NaN,member
349199,D328626730121A3B,docked_bike,2021-02-11 07:22:37,2021-02-11 07:59:04,Morris Canal,NaN,40.712418,-74.038525,NaN,NaN,member
...,...,...,...,...,...,...,...,...,...,...,...
2176707,273B1705779A2A7B,classic_bike,2023-05-09 15:39:04,2023-05-09 19:30:37,Bergen Ave & Stegman St,NaN,40.706788,-74.086736,40.70,-74.08,casual
2181567,8D7D10D23235B144,classic_bike,2023-05-27 14:29:55,2023-05-27 17:10:25,Harborside,NaN,40.719310,-74.034153,40.74,-74.04,casual
2181570,27BC7AD9933BB552,classic_bike,2023-05-29 16:05:44,2023-05-29 22:44:51,Hudson St & 4 St,NaN,40.740940,-74.028507,40.76,-74.05,casual
2181572,EE963FD1CFE05298,classic_bike,2023-05-11 17:46:20,2023-05-11 17:54:42,Hoboken Terminal - Hudson St & Hudson Pl,NaN,40.735913,-74.030353,40.75,-74.03,casual


In [8]:
# Remove rows with missing data
citi_bike_df = citi_bike_df.dropna()
citi_bike_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2216045 entries, 0 to 2223779
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   ride_id             2216045 non-null  object        
 1   rideable_type       2216045 non-null  object        
 2   started_at          2216045 non-null  datetime64[ns]
 3   ended_at            2216045 non-null  datetime64[ns]
 4   start_station_name  2216045 non-null  object        
 5   end_station_name    2216045 non-null  object        
 6   start_lat           2216045 non-null  float64       
 7   start_lng           2216045 non-null  float64       
 8   end_lat             2216045 non-null  float64       
 9   end_lng             2216045 non-null  float64       
 10  member_casual       2216045 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(5)
memory usage: 202.9+ MB


In [9]:
# Checking if there are records in which the start and end station are the same
test = citi_bike_df[citi_bike_df['start_station_name'] == citi_bike_df['end_station_name']]
test

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual
51,NA,NA,2020-01-01 08:10:35.584,2020-01-01 08:28:57.522,Hamilton Park,Hamilton Park,40.727596,-74.044247,40.727596,-74.044247,casual
84,NA,NA,2020-01-01 10:49:53.009,2020-01-01 10:58:06.624,Hilltop,Hilltop,40.731169,-74.057574,40.731169,-74.057574,casual
134,NA,NA,2020-01-01 12:13:59.663,2020-01-01 12:44:09.056,Marin Light Rail,Marin Light Rail,40.714584,-74.042817,40.714584,-74.042817,member
152,NA,NA,2020-01-01 12:48:10.086,2020-01-01 13:45:32.682,Bergen Ave,Bergen Ave,40.722104,-74.071455,40.722104,-74.071455,casual
161,NA,NA,2020-01-01 13:16:56.290,2020-01-01 13:47:54.918,Hoboken Ave at Monmouth St,Hoboken Ave at Monmouth St,40.735208,-74.046964,40.735208,-74.046964,casual
...,...,...,...,...,...,...,...,...,...,...,...
2223602,5DEEAE957C22FE7D,classic_bike,2023-05-08 10:19:39.000,2023-05-08 10:21:44.000,Riverview Park,Riverview Park,40.744319,-74.043991,40.744319,-74.043991,casual
2223603,52E971D624F40826,classic_bike,2023-05-19 08:04:57.000,2023-05-19 08:20:45.000,City Hall - Washington St & 1 St,City Hall - Washington St & 1 St,40.737360,-74.030970,40.737360,-74.030970,member
2223604,73845DCB534E8124,classic_bike,2023-05-19 08:04:06.000,2023-05-19 08:04:26.000,City Hall - Washington St & 1 St,City Hall - Washington St & 1 St,40.737360,-74.030970,40.737360,-74.030970,member
2223606,FCDE947BED0728CF,classic_bike,2023-05-12 23:18:25.000,2023-05-12 23:35:05.000,City Hall - Washington St & 1 St,City Hall - Washington St & 1 St,40.737360,-74.030970,40.737360,-74.030970,casual


## Export Data

In [11]:
# Export the dataframe to csv file for Tableau
citi_bike_df.to_csv('data/citi_bike_reduced_data1.csv')

## Data Exploration for Cleaning

In [116]:
# Note: I used this to determine that station ids change from numeric to alphanumeric in February 2020.
# As such, I removed the station ids from the dataframe

# Checking station ids for one station -
test = citi_bike_df[citi_bike_df['start_station_name'] == 'Hilltop']
test

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual
48,NA,NA,2020-01-01 07:45:48.818,2020-01-01 07:50:03.770,Hilltop,Baldwin at Montgomery,40.731169,-74.057574,40.723659,-74.064194,member
84,NA,NA,2020-01-01 10:49:53.009,2020-01-01 10:58:06.624,Hilltop,Hilltop,40.731169,-74.057574,40.731169,-74.057574,casual
315,NA,NA,2020-01-01 18:46:19.233,2020-01-01 19:29:27.048,Hilltop,Pershing Field,40.731169,-74.057574,40.742677,-74.051789,member
656,NA,NA,2020-01-02 09:38:44.837,2020-01-02 09:50:24.843,Hilltop,City Hall,40.731169,-74.057574,40.717732,-74.043845,member
658,NA,NA,2020-01-02 09:42:15.644,2020-01-02 09:51:24.016,Hilltop,Columbus Drive,40.731169,-74.057574,40.718355,-74.038914,member
...,...,...,...,...,...,...,...,...,...,...,...
2222282,3A7046C316C89CE2,electric_bike,2023-05-29 09:48:02.000,2023-05-29 09:55:30.000,Hilltop,Exchange Pl,40.731169,-74.057574,40.716366,-74.034344,casual
2222335,9F92666B26C714FE,electric_bike,2023-05-15 17:11:55.000,2023-05-15 17:18:15.000,Hilltop,Riverview Park,40.731169,-74.057574,40.744319,-74.043991,member
2222356,7E8BE4B6C62EB472,classic_bike,2023-05-29 18:30:31.000,2023-05-29 19:06:56.000,Hilltop,City Hall - Washington St & 1 St,40.731158,-74.057510,40.737360,-74.030970,member
2222537,2626A2373EA653AF,classic_bike,2023-05-15 08:49:27.000,2023-05-15 08:59:57.000,Hilltop,Riverview Park,40.731169,-74.057574,40.744319,-74.043991,member


In [84]:
citi_bike_list[0].head(2)

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,NaN,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,Exchange Place,3183,Newport Pkwy,3199,40.716247,-74.033459,40.728745,-74.032108,member
1,NaN,2018-01-01 12:06:18.039,2018-01-01 12:15:28.443,Exchange Place,3183,Newport Pkwy,3199,40.716247,-74.033459,40.728745,-74.032108,member


In [60]:
citi_bike_list[37].head(2)

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,Hoboken Ave at Monmouth St,JC105,Christ Hospital,JC034,40.735208,-74.046964,40.734786,-74.050444,member
1,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,Newport Pkwy,JC008,Marin Light Rail,JC013,40.728744,-74.032108,40.714584,-74.042817,member


In [61]:
citi_bike_list[65].head(2)

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_stationi_id
0,classic_bike,2023-06-27 16:06:27,2023-06-27 16:10:27,Hilltop,JC019,Christ Hospital,JC034,40.731115,-74.057468,40.734786,-74.050444,casual,JC019
1,classic_bike,2023-06-24 10:46:58,2023-06-24 10:55:27,Baldwin at Montgomery,JC020,Hamilton Park,JC009,40.723473,-74.064338,40.727596,-74.044247,member,JC020


### Check values in columns

In [44]:
# Initially used to determine if usertype and member_casual could be combined into one column 
print(citi_bike_list[36]['usertype'].unique())
print(citi_bike_list[37]['member_casual'].unique())
print(citi_bike_list[37]['rideable_type'].unique())

['Subscriber' 'Customer']
['member' 'casual']
['docked_bike']
